In [3]:
import sys
sys.path.append('/home/lijunzhang/multibranch/')
import pandas as pd
import numpy as np
import re
import itertools
import collections
import pickle
from statistics import mean, stdev
from scipy import stats, spatial

from main.algs_EstCon import *

In [54]:
# Information for NYUv2
data = 'NYUv2'
model = 'mobilenet'
iters = 20000 
tasks = ('segment_semantic','normal','depth_zbuffer') # task 0, 1, 2
metrics = {'segment_semantic': ['mIoU', 'Pixel Acc'],
           'normal': ['Angle Mean', 'Angle Median', 'Angle 11.25', 'Angle 22.5', 'Angle 30'],
           'depth_zbuffer': ['abs_err','rel_err','sigma_1.25','sigma_1.25^2','sigma_1.25^3']}
metrics_prop = {'mIoU': False, 'Pixel Acc': False, 
                'Angle Mean': True, 'Angle Median': True, 'Angle 11.25': False, 'Angle 22.5': False, 'Angle 30': False,
                'abs_err': True,'rel_err': True,'sigma_1.25': False,'sigma_1.25^2': False,'sigma_1.25^3': False} # True: the lower the better
reduce_var_length = {'train_loss': 1, 'val_loss': 1, 'val_acc': 1}

In [71]:
def confirm_complete(lines, iters):
    rev_lines = lines[::-1]
    for line in rev_lines:
        if 'Iter' in line:
            it = int(line.split(' ')[1])
            break
    if it != iters:
        return False
    else:
        return True

def extract_loss_results(lines, tasks, reduce_var_length, train=True):
    # Function: Extract train loss 
    loss_lst = {}
    for task in tasks:
        if train:
            loss_lst[task] = collections.deque(reduce_var_length['train_loss']*[0], reduce_var_length['train_loss'])
        else:
            loss_lst[task] = collections.deque(reduce_var_length['val_loss']*[0], reduce_var_length['val_loss'])
    
    for line in lines:
        for task in tasks:
            if task[:4] in line and ((train and 'Train Loss' in line) or (not train and 'Val Loss' in line)):
                loss = float(line.split(': ')[1])
                loss_lst[task].append(loss)
                
    final_loss = {task:-1 for task in tasks}
    for task in tasks:
        final_loss[task] = mean(loss_lst[task])
    return final_loss


def extract_metric_results(lines, tasks, metrics, metrics_prop, metrics_refer, reduce_var_length):
    # Function: Extract val metrics
    
    def rel_perf(results, refer, lower=True):
        # Function: Compute rel. perf.
        if lower:
            return (refer - np.array(results))/refer*100
        else:
            return (np.array(results) - refer)/refer*100
        
    metric_queue = {}
    for metric in metrics_prop:
        metric_queue[metric] = []

    for line in lines:
        for metric in metrics_prop:
            if "'"+metric in line:
                value = float(re.findall("\d+\.\d+", line.split(metric)[1])[0])
                metric_queue[metric].append(value)
                
    for metric in metrics_prop:
        metric_queue[metric] = rel_perf(metric_queue[metric], metrics_refer[metric], metrics_prop[metric])
        
    task_rel_perf = {}
    for task in tasks:
        temp = 0
        idx = 0
        for metric in metrics[task]:
            idx += 1
            temp += mean(metric_queue[metric][-reduce_var_length['val_acc']:])
        task_rel_perf[task] = temp/idx
    return task_rel_perf

## real train and val loss, and task rel. perf.

In [8]:
layout_idx = [0,7,11,10,9,8,16,15,39,31,49,38,48,40,17,4,1,27,6,23] # verify_0221

In [14]:
if data is 'NYUv2' and model is 'mobilenet':
    metrics_refer = {'mIoU': 0.2036, 'Pixel Acc': 0.4944,
                     'Angle Mean': 18.1772, 'Angle Median': 16.6208, 'Angle 11.25': 28.3726, 'Angle 22.5': 70.2082, 'Angle 30': 85.5802,
                     'abs_err': 0.7721, 'rel_err': 0.28, 'sigma_1.25': 47.9255, 'sigma_1.25^2': 78.4649, 'sigma_1.25^3': 92.813}

In [75]:
real_results = {'train_loss': {task: [] for task in tasks}, 
                'val_loss': {task: [] for task in tasks},
                'rel_perf': {task: [] for task in tasks}}

In [76]:
# For each layout
for idx in layout_idx:
    log = 'layout_' + str(idx) + '.stdout'
    with open('./log/layout_'+data+'_'+model+'/'+log) as f:
        lines = f.readlines()
        lines = [line.rstrip() for line in lines]
        if not confirm_complete(lines,iters):
                print(str(idx) + ' not complete')
                continue
        train_loss = extract_loss_results(lines, tasks, reduce_var_length)
        val_loss = extract_loss_results(lines, tasks, reduce_var_length, False)
        
        for task in tasks:
            real_results['train_loss'][task].append(train_loss[task])
            real_results['val_loss'][task].append(val_loss[task])
            
        print('Layout Idx: {}'.format(idx), flush=True)
        print('Train Loss: {}\nVal Loss: {}'.format(train_loss, val_loss), flush=True)
        print('-'*80)  

Layout Idx: 0
Train Loss: {'segment_semantic': 0.9127, 'normal': 0.0564, 'depth_zbuffer': 0.4817}
Val Loss: {'segment_semantic': 1.5478, 'normal': 0.0628, 'depth_zbuffer': 0.5709}
--------------------------------------------------------------------------------
Layout Idx: 7
Train Loss: {'segment_semantic': 0.8639, 'normal': 0.056, 'depth_zbuffer': 0.4753}
Val Loss: {'segment_semantic': 1.6028, 'normal': 0.0658, 'depth_zbuffer': 0.5656}
--------------------------------------------------------------------------------
Layout Idx: 11
Train Loss: {'segment_semantic': 0.8666, 'normal': 0.0559, 'depth_zbuffer': 0.4705}
Val Loss: {'segment_semantic': 1.5885, 'normal': 0.0634, 'depth_zbuffer': 0.5894}
--------------------------------------------------------------------------------
Layout Idx: 10
Train Loss: {'segment_semantic': 0.8742, 'normal': 0.0555, 'depth_zbuffer': 0.4838}
Val Loss: {'segment_semantic': 1.5718, 'normal': 0.064, 'depth_zbuffer': 0.6372}
-------------------------------------

In [77]:
# For each layout
for idx in layout_idx:
    log = 'layout_' + str(idx) + '.stdout'
    with open('./log/layout_'+data+'_'+model+'/'+log) as f:
        lines = f.readlines()
        lines = [line.rstrip() for line in lines]
        if not confirm_complete(lines,iters):
                print(str(idx) + ' not complete')
                continue
        task_rel_perf = extract_metric_results(lines, tasks, metrics, metrics_prop, metrics_refer, reduce_var_length)
        
        for task in tasks:
            real_results['rel_perf'][task].append(task_rel_perf[task])
        
        print('Layout Idx: {}'.format(idx), flush=True)
        print('Task Rel Perf: {}'.format(task_rel_perf), flush=True)
        print('-'*80)

Layout Idx: 0
Task Rel Perf: {'segment_semantic': -1.1310767034797578, 'normal': 1.4515324945649222, 'depth_zbuffer': 17.206554719771017}
--------------------------------------------------------------------------------
Layout Idx: 7
Task Rel Perf: {'segment_semantic': -2.000797934906316, 'normal': -1.4825044535798682, 'depth_zbuffer': 16.487610120740122}
--------------------------------------------------------------------------------
Layout Idx: 11
Task Rel Perf: {'segment_semantic': -1.726348859684259, 'normal': 1.254881236663984, 'depth_zbuffer': 15.993379030297154}
--------------------------------------------------------------------------------
Layout Idx: 10
Task Rel Perf: {'segment_semantic': -0.6905037798589786, 'normal': 0.9432141393187079, 'depth_zbuffer': 13.339899534132709}
--------------------------------------------------------------------------------
Layout Idx: 9
Task Rel Perf: {'segment_semantic': -2.3099381044118505, 'normal': 0.8691874595392981, 'depth_zbuffer': 10.766

## SROCC

In [86]:
print('train loss vs val loss')
for task in tasks:
    corr = stats.spearmanr(real_results['train_loss'][task],real_results['val_loss'][task])
    print('Task {}:\t {}'.format(task[:4], corr))
corr = stats.spearmanr(np.sum([real_results['train_loss'][task] for task in tasks], axis=0),
                       np.sum([real_results['val_loss'][task] for task in tasks], axis=0))
print('Overall:\t {}'.format(corr))

train loss vs val loss
Task segm:	 SpearmanrResult(correlation=0.13834586466165413, pvalue=0.5607873732874131)
Task norm:	 SpearmanrResult(correlation=-0.17233638858423728, pvalue=0.46750191037322486)
Task dept:	 SpearmanrResult(correlation=0.918796992481203, pvalue=1.0827727759514947e-08)
Overall:	 SpearmanrResult(correlation=0.7115457441893588, pvalue=0.00043453614698927293)


In [87]:
print('train loss vs rel perf')
for task in tasks:
    corr = stats.spearmanr(real_results['train_loss'][task],real_results['rel_perf'][task])
    print('Task {}:\t {}'.format(task[:4], corr))
corr = stats.spearmanr(np.sum([real_results['train_loss'][task] for task in tasks], axis=0),
                       np.mean([real_results['rel_perf'][task] for task in tasks], axis=0))
print('Overall:\t {}'.format(corr))

train loss vs rel perf
Task segm:	 SpearmanrResult(correlation=-0.47067669172932325, pvalue=0.03621455166600094)
Task norm:	 SpearmanrResult(correlation=-0.6601434392342201, pvalue=0.0015372864271503688)
Task dept:	 SpearmanrResult(correlation=-0.9383458646616539, pvalue=9.762241109318019e-10)
Overall:	 SpearmanrResult(correlation=-0.793984962406015, pvalue=2.923733879684014e-05)


In [88]:
print('val loss vs rel perf')
for task in tasks:
    corr = stats.spearmanr(real_results['val_loss'][task],real_results['rel_perf'][task])
    print('Task {}:\t {}'.format(task[:4], corr))
corr = stats.spearmanr(np.sum([real_results['val_loss'][task] for task in tasks], axis=0),
                       np.mean([real_results['rel_perf'][task] for task in tasks], axis=0))
print('Overall:\t {}'.format(corr))

val loss vs rel perf
Task segm:	 SpearmanrResult(correlation=-0.4616541353383458, pvalue=0.040457634341480954)
Task norm:	 SpearmanrResult(correlation=-0.353343929780536, pvalue=0.1264520510817935)
Task dept:	 SpearmanrResult(correlation=-0.9774436090225563, pvalue=1.322934226421056e-13)
Overall:	 SpearmanrResult(correlation=-0.8852952863751326, pvalue=2.1380395587912932e-07)
